In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpi4py import MPI
from eigentools import Eigenproblem, CriticalFinder
import dedalus.public as de
import numpy as np
import time

In [13]:
def fastest_rate(_kx,_a):
    """Constructs an Eigenproblem object that is to be solved by Dedalus 2.
   Args:
       _kx : (float) wavenumber of the perturbation
       _a : (float) activity 
   Returns:
       EP : (Eigenproblem) the corresponding eigenproblem
   """
   
    # relevant parameters
    llambda = 1
    l = 0.1
    W = 1
    tau = 1
    eta = 1
    a = _a

    # resolution
    ny = 58

    # define spectral representation of the solution
    y = de.Chebyshev('y',ny,interval=[-1,1])
    d = de.Domain([y], grid_dtype=np.complex128)

    # initialize problem
    evp = de.EVP(d, ['vx','vx1','vx2',\
                    'vy','vy1',\
                    'vz','vz1','vz2',\
                    'Qxx','Qxx1',\
                    'Qxy','Qxy1',\
                    'Qxz','Qxz1',\
                    'Qyz','Qyz1',\
                    'Qzz','Qzz1'],'sigma')

    # define paramters 
    # evp.parameters['_llambda'] = _llambda
    evp.parameters['eta'] = eta
    evp.parameters['tau'] = tau
    evp.parameters['eta'] = eta
    evp.parameters['a'] = _a
    evp.parameters['l'] = l
    evp.parameters['W'] = W
    evp.parameters['ell_sq'] = l**2 / W**2
    evp.parameters['kx'] = _kx
    
    # Order parameter equations
    evp.add_equation("Qxx + ell_sq*(kx**2)*Qxx - 2*1j*kx*vx + Qxx*sigma*tau - ell_sq*dy(Qxx1) = 0")
    evp.add_equation("Qxy + ell_sq*(kx**2)*Qxy - 1j*kx*vy + Qxy*sigma*tau - vx1 - ell_sq*dy(Qxy1) = 0")
    evp.add_equation("ell_sq*(kx**2)*Qxz + Qxz*(1+sigma*tau) - 1j*kx*tau*vz/W - ell_sq*dy(Qxz1) = 0")
    evp.add_equation("Qyz + ell_sq*(kx**2)*Qyz + Qyz*sigma*tau - ell_sq*dy(Qyz1) - tau*vz1 = 0")
    evp.add_equation("ell_sq*(kx**2)*Qzz + Qzz*(1 + sigma*tau) - ell_sq*dy(Qzz1) = 0")
    
    # velocity equations
    evp.add_equation("1j*a*kx*Qxz1 + a*dy(Qyz1) + (kx**2)*eta*vz1 - (W**2)*eta*dy(vz2) = 0")
    evp.add_equation("-1j*(kx**3)*vz/W + kx*(a*kx*Qxz - 1j*a*W*Qyz1 + 1j*W*eta*dy(vz1)) / eta = 0")
    evp.add_equation("-1j*(kx**3)*vy*eta + a*(kx**2)*Qxy*tau + 2j*a*kx*tau*Qxx1 + 1j*a*kx*tau*Qzz1"+ 
                     "+ a*tau*dy(Qxy1) + kx*eta*(kx*vx1 + 1j*dy(vy)) - (W**2)*eta*tau*dy(vx) = 0")
    # Auxiliary equation for formulating the problem as first-order
    evp.add_equation("dy(vx) - vx1 = 0")
    evp.add_equation("dy(vx1) - vx2 = 0")
    evp.add_equation("dy(vy) - vy1 = 0")
    evp.add_equation("dy(vz) - vz1 = 0")
    evp.add_equation("dy(vz1) - vz2 = 0")
    evp.add_equation("dy(Qxx) - Qxx1 = 0")
    evp.add_equation("dy(Qxy) - Qxy1 = 0")
    evp.add_equation("dy(Qxz) - Qxz1 = 0")
    evp.add_equation("dy(Qyz) - Qyz1 = 0")
    evp.add_equation("dy(Qzz) - Qzz1 = 0")
    # !!! Add incompressibility constraint?? !!!!
    evp.add_equation("left(vy) = 0")
    evp.add_equation("right(vy) = 0")
    
    # impose boundary conditions
    evp.add_bc("left(vx) = 0")
    evp.add_bc("right(vx) = 0")
    evp.add_bc("left(vy) = 0")
    evp.add_bc("right(vy) = 0")
    evp.add_bc("left(vz) = 0")
    evp.add_bc("right(vz) = 0")
    evp.add_bc("left(Qxx) = 0")
    evp.add_bc("right(Qxx) = 0")
    evp.add_bc("left(Qxy) = 0")
    evp.add_bc("right(Qxy) = 0")
    evp.add_bc("left(Qxz) = 0")
    evp.add_bc("right(Qxz) = 0")
    evp.add_bc("left(Qyz) = 0")
    evp.add_bc("right(Qyz) = 0")
    evp.add_bc("left(Qzz) = 0")
    evp.add_bc("right(Qzz) = 0")
    # Need two more boundary conditions here?

    # create eigenproblem object
    f_func = lambda z: z.imag
    g_func = lambda z: z.real
    EP = Eigenproblem(evp, grow_func=g_func, freq_func=f_func)
    return EP

# Here is how to use the growth_rate function :)
#     rate, indx, freq = EP.growth_rate(sparse=False)
#     print("fastest growing mode: {} @ freq {}".format(rate, freq))
#     return rate

In [3]:
fastest_rate(1.25, 3)
# Change the code from np.int to just int!
# ~/anaconda3/envs/dedalus2/lib/python3.11/site-packages/eigentools/eigenproblem.py:707, in Eigenproblem._discard_spurious_eigenvalues(self)

2023-04-29 01:00:20,948 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2023-04-29 01:00:21,024 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
fastest growing mode: 1.984375000000008 @ freq 8.91036105604159e-16


1.984375000000008

In [5]:
fastest_rate(1.25, 0.01)

2023-04-29 01:03:32,902 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2023-04-29 01:03:32,984 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
fastest growing mode: -1.0056249999999953 @ freq 1.0208621624523362e-16


-1.0056249999999953

In [7]:
fastest_rate(1e-5, 1)

2023-04-29 01:05:47,453 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2023-04-29 01:05:47,526 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
fastest growing mode: -1.0000000000009928 @ freq 4.0046033152150255e-16


-1.0000000000009928